In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 

In [3]:
import warnings
warnings.filterwarnings('ignore') #ignore ipython notebook warnings

In [4]:
dataset_loction = "/Users/sandylee/Google Drive/Flatiron School/london-ds-010620/mod_1/student/KayCee/source/kc_house_data.csv" #variable for dataset location

In [5]:
data = pd.read_csv(dataset_loction) #import dataset

In [6]:
duplicate_properties = data[data.duplicated(subset="id",keep=False)] #extract properties sold more than once into a new dataframe

In [7]:
duplicate_properties.insert(2,"time_diff",0) #add a column for time difference

In [8]:
duplicate_properties.insert(4,"price_diff",0) #add a column for price difference

In [9]:
duplicate_properties.insert(5,"rate_of_price_increase",0) #add a column for the rate of price increase

In [10]:
duplicate_properties["date"] = pd.to_datetime(duplicate_properties['date']) #convert date column to datetime objects from str

In [11]:
duplicate_properties["time_diff"] = pd.to_datetime(duplicate_properties['time_diff']) #convert time_diff column to datetime objects from str

In [12]:
duplicate_properties = duplicate_properties.reset_index() #reset index 

In [13]:
duplicate_properties = duplicate_properties.drop("index", 1) #drop duplicate index column

In [14]:
duplicate_properties = duplicate_properties.drop(317) #drop one entry from a property sold 3 times that is causing issues

In [15]:
duplicate_properties = duplicate_properties.reset_index() #reset index (again) 

In [16]:
duplicate_properties = duplicate_properties.drop("index", 1) #drop duplicate index column (again)

In [17]:
duplicate_properties['time_diff'] = duplicate_properties.groupby(['id'])['date'].shift(1) #run a lag operation on dates

In [18]:
duplicate_properties['price_diff'] = duplicate_properties.groupby(['id'])['price'].shift(1) #run a lag operation on prices

In [19]:
duplicate_properties["time_diff"] = duplicate_properties['date'] - duplicate_properties['time_diff'] #get the date difference and add it to 'time_diff' column

In [20]:
duplicate_properties["price_diff"] = duplicate_properties['price'] - duplicate_properties['price_diff'] #get the price difference and add it to 'price_diff' column

In [21]:
analysis_df = duplicate_properties.copy() #make a new dataframe for analysis

In [22]:
for row in range(0,len(analysis_df),2):
     analysis_df.drop(index=row,inplace=True) #remove redundant NaN rows 

In [23]:
analysis_df.insert(3,"time_diff_seconds",0) #add a column for time difference as an int

In [24]:
analysis_df["time_diff_seconds"] = pd.to_numeric(analysis_df["time_diff"])/1000000000 #convert timedelta obeject to seconds (stored as ns by default)

In [25]:
analysis_df["rate_of_price_increase"] = analysis_df["price_diff"]/(analysis_df["time_diff_seconds"]/86400) #calculate rate of increase in prices

In [26]:
hot_zip_codes = analysis_df.groupby(["zipcode"])["rate_of_price_increase"].mean() #create a Pandas series of rate of the mean rate of price inrease grouped by zip code

In [27]:
hot_zip_codes = analysis_df.groupby(["zipcode"]).mean() #group everything by zipcode and take the mean

In [28]:
hot_zip_codes.drop(columns = ["id", "time_diff_seconds", "price", "price_diff", "bedrooms", "bathrooms"],inplace = True) #remove extranous columns

In [29]:
hot_zip_codes.drop(columns = ["sqft_living", "sqft_lot", "floors", "waterfront", "view", "condition"],inplace = True) #remove extranous columns

In [30]:
hot_zip_codes.drop(columns = ["grade", "sqft_above", "yr_built", "yr_renovated", "sqft_living15", "sqft_lot15"],inplace = True) #remove extranous columns

In [31]:
hot_zip_codes.sort_values(ascending = False, inplace = True, by="rate_of_price_increase")  #sort hot zip codes in descending order

In [33]:
lat_long = hot_zip_codes[["lat", "long"]].copy()

In [34]:
import folium

In [35]:
locationlist = lat_long.values.tolist()

In [37]:
map = folium.Map(location=[47.530, -122.200],zoom_start=10)
for point in range(0,len(locationlist)):
    folium.CircleMarker(locationlist[point],radius=1).add_to(map)

In [38]:
map